In [19]:
import cv2
import mediapipe as mp
from cvzone.FaceDetectionModule import FaceDetector
import pyfirmata
import numpy as np
import time

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

In [6]:
def shoot(image):
    cv2.putText(image, 'Shooting', (50, 100), cv2.FONT_HERSHEY_PLAIN, 2, (255,255, 0), 2)

In [33]:
# port = "COM7"
# board = pyfirmata.Arduino(port)
# servo_pinX = board.get_pin('d:9:s') #pin 9 Arduino
# servo_pinY = board.get_pin('d:10:s') #pin 10 Arduino

cap = cv2.VideoCapture(0)
# ws, hs = 1280, 720
# cap.set(3, ws)
# cap.set(4, hs)

# detector = FaceDetector()
servoPos = [90, 90]  # initial servo position
offsetY = 150
start_time = None

WARN_TIMER = 6
COOLDOWN_TIMER = 6


with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        success, image = cap.read()

        if not success:
            print("Camera not found.")
            print("Please check whether the camera is connected properly.")
            # If loading a video, use 'break' instead of 'continue'.
            break
        

        isSurrender = False
        isShooting = False
        surrenderTextColor = (0, 0, 255)


        # Pose detection
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

        if results.pose_landmarks:


            left_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST].y
            right_wrist_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST].y

            right_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER].y
            left_shoulder_y = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER].y

            nose_y = int(
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].y * image.shape[0] + offsetY)
            nose_x = int(
                results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].x * image.shape[1])

            if left_wrist_y < left_shoulder_y and right_wrist_y < right_shoulder_y:
                isSurrender = True
                surrenderTextColor = (0, 255, 0)

            if nose_x and nose_y:
                # convert coordinat to servo degree
                servoX = np.interp(nose_x, [0, image.shape[0]], [0, 180])
                servoY = np.interp(nose_y, [0, image.shape[1]], [0, 180])

                if servoX < 0:
                    servoX = 0
                elif servoX > 180:
                    servoX = 180
                if servoY < 0:
                    servoY = 0
                elif servoY > 180:
                    servoY = 180

                servoPos[0] = servoX
                servoPos[1] = servoY

                cv2.circle(image, (nose_x, nose_y), 50, (0, 0, 255), 2)
                cv2.circle(image, (nose_x, nose_y),
                           15, (0, 0, 255), cv2.FILLED)
                
                if not isSurrender:
                    if start_time == None:
                        print("Starting timer...")
                        start_time = time.time()

                    curr_time = time.time()

                    if curr_time - start_time > WARN_TIMER:
                        isShooting = True
                    else:
                        cv2.putText(image, f'Shooting in: {int(WARN_TIMER - (curr_time - start_time))}', (50, 100),
                                    cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 0), 2)
                    
                else:
                    isShooting = False
                    start_time = None
                

        if isShooting:
            shoot(image)

        cv2.putText(image, f'Surrender: {isSurrender}', (50, 50),
                    cv2.FONT_HERSHEY_PLAIN, 2, surrenderTextColor, 2)

        # Moving servos
        # servo_pinX.write(servoPos[0])
        # servo_pinY.write(servoPos[1])

        print(start_time)

        # Showing image
        # cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break

cv2.destroyAllWindows()
cap.release()


Starting timer...
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
1676043252.1462662
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
Starting timer...
1676043256.0513334
1676043256.0513334
1676043256.0513334
1676043256.0513334
1676043256.0513334
16760432

In [ ]:
    # Face detection
    # image, bboxs = detector.findFaces(image, draw=False)
    # if bboxs:
    #     #get the coordinate
    #     fx, fy = bboxs[0]["center"][0], bboxs[0]["center"][1] + offsetY
    #     pos = [fx, fy]

    #     # #convert coordinat to servo degree
    #     # servoX = np.interp(fx, [0, ws], [0, 180])
    #     # servoY = np.interp(fy, [0, hs], [0, 180])

    #     # if servoX < 0:
    #     #     servoX = 0
    #     # elif servoX > 180:
    #     #     servoX = 180
    #     # if servoY < 0:
    #     #     servoY = 0
    #     # elif servoY > 180:
    #     #     servoY = 180

    #     # servoPos[0] = servoX
    #     # servoPos[1] = servoY

    #     cv2.circle(image, (fx, fy), 50, (0, 0, 255), 2)
    #     # cv2.putText(image, str(pos), (fx+15, fy-15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2 )
    #     # cv2.line(image, (0, fy), (ws, fy), (0, 0, 0), 2)  # x line
    #     # cv2.line(image, (fx, hs), (fx, 0), (0, 0, 0), 2)  # y line
    #     cv2.circle(image, (fx, fy), 15, (0, 0, 255), cv2.FILLED)
    #     cv2.putText(image, "Target Found", (50, 80), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2 )

    # else:
    #     cv2.putText(image, "No Target", (50, 80), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2 )
    #     # cv2.circle(image, (640, 360), 80, (0, 0, 255), 2)
    #     # cv2.circle(image, (640, 360), 15, (0, 0, 255), cv2.FILLED)
    #     # cv2.line(image, (0, 360), (ws, 360), (0, 0, 0), 2)  # x line
    #     # cv2.line(image, (640, hs), (640, 0), (0, 0, 0), 2)  # y line

    # servo_pinX.write(servoPos[0])
    # servo_pinY.write(servoPos[1])

    # with mp_face_detection.FaceDetection(
    #     model_selection=1, min_detection_confidence=0.5) as face_detection:

    #   image.flags.writeable = False
    #   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #   face_results = face_detection.process(image)

    #   image.flags.writeable = True
    #   image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #   if face_results.detections:
    #     cv2.putText(image, "Target Found", (50, 80), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2 )
    #     for detection in face_results.detections:
    #       mp_drawing.draw_detection(image, detection)
    #   else:
    #     cv2.putText(image, "No Target", (50, 80), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2 )